In [43]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable #自動微分用
import numpy as np

import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
import util

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [76]:
import pickle
emoji_index = pickle.loads(open('./emoji_index.pkl', 'rb').read())
def emojiFromIndex(num):
    return [k for k, v in emoji_index.items() if v == num]

emojiFromIndex(10)

['😅']

In [46]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [47]:
import csv
def readLangs(lang1, lang2, fn, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(fn, encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[s for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [48]:
MAX_LENGTH = 31

In [49]:
def prepareData(lang1, lang2, fn, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, fn, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [50]:
input_lang, output_lang, pairs = prepareData('in', 'out', 'text_label_split.tsv', False)
print(random.choice(pairs))

input_lang_test, output_lang_test, pairs_test = prepareData('in', 'out', 'text_label_split2.tsv', False)
print(random.choice(pairs_test))

Reading lines...
Read 530706 sentence pairs
Trimmed to 530706 sentence pairs
Counting words...
Counted words:
in 107663
out 22
['昨日 フォロワー に なっ た 方 ﾃﾞｽ  ', '5']
Reading lines...
Read 1045 sentence pairs
Trimmed to 1045 sentence pairs
Counting words...
Counted words:
in 2376
out 22
['この 時期 に なる と 夜空 は 綺麗 です なぁ ', '8']


In [51]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [52]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [53]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [54]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [55]:
teacher_forcing_ratio = 0.5

In [56]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [57]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [58]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses, "losses.png")

In [59]:
import matplotlib.pyplot as plt
#plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points, filename):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig(filename)
    plt.show()
    plt.close()

In [79]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd

def showConfusionMatrix(y_test, y_pred):
    conf_mat = confusion_matrix(y_test, y_pred)
    index = columns = [str(l) for l in range(len(emoji_index))]
    df = pd.DataFrame(conf_mat, index=index, columns=columns)
    
    fig = plt.figure(figsize = (7,7))
    sns.heatmap(df, annot=False, square=True, fmt='.0f', cmap="Blues")
    plt.title('20 emojis classification')
    plt.xlabel('emojis')
    plt.ylabel('prediction')
    fig.savefig("confusion_matrix_torch3.png")
    plt.show()
    plt.close()

In [61]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [62]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        print('original emoji = ', emojiFromIndex(int(pair[1])))
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('output emoji < ', emojiFromIndex(int(output_words[0])))
        print('')

In [73]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

def evaluateTestdata(encoder, decoder):
    y_test, y_pred = [], []
    for pair in pairs_test:
        y_test.append(int(pair[1]))
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        y_pred.append(int(output_words[0]))
    print("precision", precision_score(y_test, y_pred, average=None))
    print("report", classification_report(y_test, y_pred))
    print("accuracy score", accuracy_score(y_test, y_pred))
    showConfusionMatrix(y_test, y_pred)
    

In [64]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)

In [65]:
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

In [66]:
trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 42s (- 23m 59s) (5000 6%) 1.4156
3m 23s (- 22m 5s) (10000 13%) 1.3294
5m 5s (- 20m 20s) (15000 20%) 1.2860
6m 46s (- 18m 37s) (20000 26%) 1.2747
8m 27s (- 16m 54s) (25000 33%) 1.2529
10m 7s (- 15m 11s) (30000 40%) 1.2316
11m 49s (- 13m 30s) (35000 46%) 1.2196
13m 30s (- 11m 49s) (40000 53%) 1.2261
15m 12s (- 10m 8s) (45000 60%) 1.2207
16m 53s (- 8m 26s) (50000 66%) 1.2111
18m 33s (- 6m 44s) (55000 73%) 1.1981
20m 14s (- 5m 3s) (60000 80%) 1.2105
21m 55s (- 3m 22s) (65000 86%) 1.1995
23m 36s (- 1m 41s) (70000 93%) 1.1951
25m 18s (- 0m 0s) (75000 100%) 1.1924


In [67]:
evaluateRandomly(encoder1, attn_decoder1)

> やっぱり 、 ジャズ って 楽しい 
= 6
original emoji =  ['😊']
< 6 <EOS>
output emoji <  ['😊']

> いま 野菜 に なっ てる よ  
= 3
original emoji =  ['😂']
< 4 <EOS>
output emoji <  ['💕']

> 前日 とか 空気 悪 すぎ て どう なる か と 思っ た けど 金賞 とれ て よかっ た 
= 6
original emoji =  ['😊']
< 3 <EOS>
output emoji <  ['😂']

> しょ ～ ゆ ～ 事 です な 
= 15
original emoji =  ['😉']
< 18 <EOS>
output emoji <  ['💦']

>  彼氏 彼女 い そう お るん
= 2
original emoji =  ['😍']
< 2 <EOS>
output emoji <  ['😍']

> 普段 から し ない と なー 
= 10
original emoji =  ['😅']
< 3 <EOS>
output emoji <  ['😂']

> 素敵 な 一 日 に なり ます よう に 、 心から 願っ て い ます 
= 19
original emoji =  ['😆']
< 8 <EOS>
output emoji <  ['✨']

> バスケ の マネージャー し て はっ て めっちゃ かっこいい です 
= 8
original emoji =  ['✨']
< 3 <EOS>
output emoji <  ['😂']

> 見 れる の 楽しみ だ な～ 
= 6
original emoji =  ['😊']
< 8 <EOS>
output emoji <  ['✨']

> 知り合っ て 時間 の 人 と 鬼 仲良く なる って なに よ 
= 3
original emoji =  ['😂']
< 3 <EOS>
output emoji <  ['😂']



In [68]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "仲良く し て")
plt.matshow(attentions.numpy())
plt.savefig('frendme.png')
plt.close()

In [69]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    #plt.show()
    plt.savefig('frendme.png')
    plt.close()

In [70]:
def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    print('output emoji = ', emojiFromIndex(int(output_words[0])))
    showAttention(input_sentence, output_words, attentions)

In [71]:
evaluateAndShowAttention("なお クン は 気圧 の 変化 に も 弱い から 、 いろいろ 心配 で あり やす 、 、 、")

input = なお クン は 気圧 の 変化 に も 弱い から 、 いろいろ 心配 で あり やす 、 、 、
output = 8 <EOS>
output emoji =  ['✨']


In [80]:
evaluateTestdata(encoder1, attn_decoder1)

precision [0.33333333 0.         0.09174312 0.23809524 0.         0.2406015
 0.         0.27078385 0.33333333 0.25       0.425      0.23529412
 0.2        0.         0.         0.5        0.54166667 0.36363636
 0.         0.        ]
report              precision    recall  f1-score   support

          1       0.33      0.08      0.13        24
          2       0.00      0.00      0.00        22
          3       0.09      0.44      0.15        45
          4       0.24      0.19      0.21        77
          5       0.00      0.00      0.00         8
          6       0.24      0.26      0.25       122
          7       0.00      0.00      0.00         3
          8       0.27      0.58      0.37       196
          9       0.33      0.27      0.30        15
         10       0.25      0.02      0.04        84
         11       0.42      0.37      0.40        46
         12       0.24      0.14      0.18        28
         13       0.20      0.07      0.11        54
         14     

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
plt.close('all')